In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-
import time
from keras import metrics
import os
import numpy as np
import keras
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN,Activation
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec

Using TensorFlow backend.


In [3]:
input_size = int(33599 / 3)
print(input_size, 33599 / 3)

11199 11199.666666666666


In [4]:
import pandas as pd

TRAINING_PATH = './data/training/'
TESTING_PATH = './data/testing/'

categories = [dirname for dirname in os.listdir(TRAINING_PATH) if dirname[-4:] != '_cut']
print(len(categories), str(categories))

category2idx = {'Japan_Travel': 0, 'KR_ENTERTAIN': 1, 'Makeup': 2, 'Tech_Job':  3, 'WomenTalk': 4,
                  'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}


train_list = []
for category in categories:
    category_idx = category2idx[category]
    category_path = TRAINING_PATH + category + '_cut/'
    
    for filename in os.listdir(category_path):
        filepath = category_path + filename
        
        with open(filepath, encoding='utf-8') as file:
            words = file.read().strip().split(' / ')
            train_list.append([words, category_idx])
            

10 ['Tech_Job', 'e-shopping', 'babymother', 'joke', 'Makeup', 'Japan_Travel', 'WomenTalk', 'movie', 'KR_ENTERTAIN', 'graduate']


In [5]:
# for sss in train_list:
#     if sss[1] == 5:
#         print(sss)
#         break

In [6]:
test_list = []
category_path = TESTING_PATH 
    
for filename in os.listdir(category_path):
        filepath = category_path + filename
        
        with open(filepath, encoding='utf-8') as file:
            words = file.read().strip().split(' / ')
            test_list.append([words, 1])

# test_list[1]

In [7]:
# train_list[0][1]
train_dict =  dict(enumerate(train_list))
test_dict =  dict(enumerate(test_list))
# train_dict[0][1]
# len(train_dict)

In [8]:
# def getArticleVec(article):
#     article_vec = []
#     length = 0
#     for word in article[0]:
#         if word in wvv.keys():
# #             print(answer[word])
#             article_vec.append(answer[word])
#             length+=1
#         else:
#             temp = np.zeros(250)
#             article_vec.append(temp)
# #     print(len(article_vec), article_vec[0], article_vec)
#     return np.mean(article_vec ,axis=0)

# res1 = getArticleVec(train_list[0])
# # print(res1)
# print(len(res1))

In [9]:
# label_list = np.zeros((len(train_dict), 10))
# # inss = 0
# for i,article in train_dict.items():
# #     print(i,article[1])
# #     if inss > 10:
# #         break
#     sec_i = article[1]
#     label_list[i][sec_i] = 1
#     print(label_list[i])

# #     inss += 1
# print(len(label_list))

# label_new_list= np.zeros((len(train_dict), 10))
# label_list_i = 0
# for label in label_list:
# #     label_list[label_list_i] = np.reshape( label_list[label_list_i] ,(1, 10))
    
#     label_new = np.reshape(label_list[label_list_i],(1, 10))
# #     print(label_new.shape)
# #     label_list[label_list_i] = label_new
# #     print(label_list[label_list_i].shape)
#     label_new_list[label_list_i] = label_new
#     label_list_i += 1
    
# print(label_new_list.shape)
# print(label_new_list[0].shape)

In [10]:
answer = word2vec.Word2Vec.load("word2vec1.model")
print(type(answer))

word_vectors = answer.wv
wvv = word_vectors.vocab
# wvv_keys = wvv.keys()
# wvv_keys_list = list(wvv_keys)
# wvv_new_dict = dict(enumerate(wvv_keys_list))
# word_index = len(wvv_keys_list)

<class 'gensim.models.word2vec.Word2Vec'>


In [11]:
train_pickle_df = pd.read_pickle('train.pkl')
train_texts = train_pickle_df["text"]
train_texts = train_texts.values
print(len(train_texts))
print(len(wvv))
# ss = type(pickle_df["text"])
# print(ss)
# print(type(val))
ssss=0
max_ss_len = 0
for i,word in train_dict.items():
#     if ssss > 4:
#         break
    ssss += 1
    ss_len = len(word[0])
    if max_ss_len < ss_len:
        max_ss_len = ss_len
#     print(i, ss_len)
print(max_ss_len)

9000
36132
3971


In [51]:
texts = train_texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print("Found %s unique tokens" % len(word_index))
# data = pad_sequences(sequences)
# print("Shape of data tensor:" , data.shape)
max_doc_word_length = 3971
word_index = pad_sequences(sequences, maxlen=max_doc_word_length, padding='post')
# print(padded_docs)


embeddings_index = dict()
MAX_NB_WORDS = len(word_index)
for word,vocab_obj in wvv.items():
    if int(vocab_obj.index) < MAX_NB_WORDS and word == -1:
        embeddings_index[word] = word_vectors[word]
# del answer, word_vectors

print("Found %s word vectors." % len(embeddings_index))
OUT_SIZE = 250

def getArticleMatrix(article_temp):
    res = np.zeros((3971,  OUT_SIZE))
    i = 0
    for word in article_temp:
        word_embedding_vector = embeddings_index.get(word)
        if word_embedding_vector is not None:
            res[i] = word_embedding_vector
            res[i]=np.reshape(res[i],(1, 250))
    return res

Found 232236 unique tokens
Found 0 word vectors.


In [13]:
print(len(train_dict))
# print(train_dict[0][0])

# ARTICLES_LEN = len(train_dict)
ARTICLES_LEN = 1000
articles_matrix = np.zeros((ARTICLES_LEN, 3971, 250))
for i,article in train_dict.items():
    if i < ARTICLES_LEN:
        temp_vec = getArticleMatrix(article[0])
        if temp_vec is not None:
            articles_matrix[i] = temp_vec
    else:
        break
print(len(articles_matrix))

9000
1000


In [44]:
articles_num = len(train_dict)

embedding_matrix = np.zeros( (1,len(word_index), 250))

for i,word in word_index.items():
    embedding_vector = embeddings_index.get(word)
#     print(embedding_vector.shape)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_matrix_len = len(embedding_matrix)

print(embedding_matrix_len, OUT_SIZE, 3971)
embedding_layer = Embedding(embedding_matrix_len, 250,
                            weights = [embedding_matrix],
                            input_length = 3971,
                            trainable = False)
print("articles_num", articles_num)
print("embedding_matrix", len(embedding_matrix))

1 250 3971
articles_num 9000
embedding_matrix 1


In [46]:
print(len(embedding_matrix[0]))
# input_array = np.random.randint(embedding_matrix_len, size=(50, 3971))
# print(input_array.shape)

232236
(50, 3971)


In [31]:
embedding_matrix_len = len(embedding_matrix[0])
label_list = np.zeros((embedding_matrix_len, 10))
label_id = 0
for i,article in train_dict.items():
    article_len = len(article)
    sec_i = article[1]
    for w_i in range(0, article_len):
        label_list[label_id][sec_i] = 1
        label_id += 1
#             break
print(label_list[i])
    
# label_new_list= np.zeros((embedding_matrix_len, 10))
# label_list_i = 0
# for label in label_list:
#     label_new = np.reshape(label_list[label_list_i],(1, 10))
#     label_new_list[label_list_i] = label_new
#     label_list_i += 1
print(len(label_list))    
# del label_list    
# print(label_list.shape)
# print(label_new_list[0].shape)

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
232236


In [32]:
# input_dim = 400
# output_dim = 100
model = Sequential()
model.add(embedding_layer)

model.add(SimpleRNN(input_dim = 250, output_dim = 50, unroll=True))
nb_classes = 10
model.add(Dense( nb_classes, input_dim = 3971))
model.add(Activation('softmax'))
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=[metrics.mae, metrics.categorical_accuracy])
model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 3971, 250)         250       
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 50)                15050     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 15,810
Trainable params: 15,560
Non-trainable params: 250
_________________________________________________________________


In [49]:
train_result= articles_matrix[0]
train_label = np.asarray(label_list[0])
train_result= np.reshape( train_result ,(250, 3971))
train_label= np.zeros((1, 10))

print(train_result.shape)
print(train_label)
print(train_label.shape)


(250, 3971)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(1, 10)


In [50]:
history = model.fit(x = train_result, y = train_label, 
                    validation_split=0.1, 
                    callbacks=[early_stopping], 
                    epochs = 1, verbose = 1)

ValueError: Input arrays should have the same number of samples as target arrays. Found 250 input samples and 1 target samples.

In [ ]:
test_feature=  embedding_matrix[5010:5011]
predict_res = model.predict(test_feature, batch_size= 32, verbose=0)
print(len(predict_res), predict_res)

final_res = []
now_item = predict_res[0]
max_val = max(now_item)
final_res.append(np.argmax(now_item))
print(final_res)

In [ ]:
# del model
# model = load_model('my_model.h5')
len(embedding_matrix)

In [ ]:
# final_res = []
# print(len(test_embedding_matrix))
# for predicc in test_embedding_matrix:
#     test_feature= np.asarray(predicc)
#     predict_res = model.predict(test_feature, batch_size= 32, verbose=0)
# #     print(len(predict_res), predict_res[0])
    
#     now_item = predict_res[0]
#     max_val = max(now_item)
#     final_res.append(np.argmax(now_item))
# print(final_res)

In [ ]:
# # result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
# print(len(final_res))
# result_txt = "result001" + ".txt"
# ids = 0
# with open(result_txt, 'a') as out:
#     out.write("id,category" + '\n')
#     for value in final_res:
#         out.write(str(ids) + "," + str(value) + '\n')
#         ids += 1

In [ ]:
# start_time = time.time()
# start = 0

# for embed_item_index in range(0, 487):
# #     train_result= np.asarray([embedding_matrix])
# #     train_label =  np.asarray([label_list[label_id]])
    
#     end = start + 100
#     train_result= articles_matrix[start:end]
#     train_label = label_list[start:end]
#     history = model.fit(x = train_result, y = train_label, validation_split=0.1, callbacks=[early_stopping], epochs = 5, verbose = 0)
#     model.save('my_model.h5')
#     scores = model.evaluate(x = train_result, y = train_label, batch_size=32, verbose = 0)
#     if  embed_item_index % 10 == 0 :  
#         end_time = time.time()
#         excution_time = end_time- start_time
#         excution_time_min = round(excution_time / 60, 2)
#         print(embed_item_index, " , loss : ", round(scores[0], 6), "   acc : " , scores[1],
#               "  minutes: " , excution_time_min)
# #     loss_func_name = 'categorical_crossentropy'
# # #     for value in history.history[loss_func_name]:
# #     print(history)
        
#     if embed_item_index < 40 :  
#         pass
        
#     else:
#         break
#     start += 100